In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/hitters.csv')

In [3]:
%matplotlib inline

In [5]:
df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


In [7]:
from sklearn.tree import DecisionTreeRegressor

In [8]:
tree = DecisionTreeRegressor(random_state = 1, max_depth = 4)

In [9]:
tree

DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=1, splitter='best')

In [10]:
tree.get_params()

{'criterion': 'mse',
 'max_depth': 4,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': False,
 'random_state': 1,
 'splitter': 'best'}

In [12]:
df['Salary'].fillna(df.Salary.mean(),inplace = True)

In [13]:
X = df.loc[:, df.columns !='Salary']

In [14]:
y = df['Salary']

In [15]:
X = pd.get_dummies(X)

In [17]:
X.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,...,CWalks,PutOuts,Assists,Errors,League_A,League_N,Division_E,Division_W,NewLeague_A,NewLeague_N
0,293,66,1,30,29,14,1,293,66,1,...,14,446,33,20,1,0,1,0,1,0
1,315,81,7,24,38,39,14,3449,835,69,...,375,632,43,10,0,1,0,1,0,1
2,479,130,18,66,72,76,3,1624,457,63,...,263,880,82,14,1,0,0,1,1,0
3,496,141,20,65,78,37,11,5628,1575,225,...,354,200,11,3,0,1,1,0,0,1
4,321,87,10,39,42,30,2,396,101,12,...,33,805,40,4,0,1,1,0,0,1


In [19]:
X.columns

Index(['AtBat', 'Hits', 'HmRun', 'Runs', 'RBI', 'Walks', 'Years', 'CAtBat',
       'CHits', 'CHmRun', 'CRuns', 'CRBI', 'CWalks', 'PutOuts', 'Assists',
       'Errors', 'League_A', 'League_N', 'Division_E', 'Division_W',
       'NewLeague_A', 'NewLeague_N'],
      dtype='object')

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 2019)

In [23]:
tree.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=1, splitter='best')

In [27]:
pip install graphviz

Note: you may need to restart the kernel to use updated packages.


In [50]:
from sklearn.tree import export_graphviz
import IPython
import graphviz
import re

def draw_tree(t, df, size=10, ratio=0.6, precision=0):
    """ Draws a representation of a random forest in IPython.
    Parameters:
    -----------
    t: The tree you wish to draw
    df: The data used to train the tree. This is used to get the names of the features.
    """
    s=export_graphviz(t, out_file=None, feature_names=df.columns, filled=True,
                      special_characters=True, rotate=True, precision=precision)
    IPython.display.display(graphviz.Source(re.sub('Tree {',
       f'Tree {{ size={size}; ratio={ratio}', s)))

In [ ]:
draw_tree(tree, X_train)

In [30]:
from sklearn.model_selection import cross_val_score

In [31]:
scores = cross_val_score(estimator=tree, X=X_train, y=y_train, cv=10)

In [33]:
tree.get_params()

{'criterion': 'mse',
 'max_depth': 4,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': False,
 'random_state': 1,
 'splitter': 'best'}

In [34]:
import numpy as np

In [ ]:
alphas = [1,5, 10, 25, 50]

tree_scores = []

for alpha in alphas:
    tree.set_params(min_samples_leaf=alpha)
    scores = cross_val_score(estimator=tree, X=X_train, y=y_train, cv=10)
    tree_scores.append((np.mean(scores), alpha))

In [42]:
tree_scores

[(0.16829546329660883, 1),
 (0.22146746597359016, 5),
 (0.2201189988069006, 10),
 (0.34093901108823016, 25),
 (0.30512582926340126, 50)]

In [43]:
max(tree_scores)

(0.34093901108823016, 25)

In [44]:
### bootstrap

In [45]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

In [46]:
rf.fit(X,y)

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [47]:
rf.n_estimators

10

In [ ]:
rf.estimators_

In [ ]:
alphas = [1,5, 10, 25, 50]

tree_scores = []

for i in alphas:
    tree.set_params(min_samples_leaf=alpha)
    scores = cross_val_score(estimator=tree, X=X_train, y=y_train, cv=10)
    tree_scores.append((np.mean(scores), alpha))

In [62]:
X.shape[1]

22

In [79]:
features_m = [.5, 'log2','sqrt' ]
tree_scores = []
for i in features_m:
    rf.set_params(max_features=.5)
    scores = cross_val_score(estimator=rf, X=X_train, y=y_train, cv=10)
    tree_scores.append((np.mean(scores), i))

In [80]:
tree_scores

[(0.37697239063031607, 0.5),
 (0.41981990704260175, 'log2'),
 (0.36746653482897507, 'sqrt')]